In [ ]:
import datetime
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
import os
os.chdir("G:\Edwiser material\Project\Santandarcustomer problems\Edwiser project\Azuredeployment")

Import Ml Azur SDK models

In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

Check Azure ML SDK version

In [ ]:
print(azureml.core.VERSION)

In [ ]:
ws=Workspace.create(name='santander',
                   subscription_id='eb12695a-2ddd-4326-aefd-3bdeef37cc2c',
                   resource_group='web',
                   create_resource_group=True,
                   location='southeastasia'
                   )

Write configuration to local file

In [ ]:
ws.write_config()

Access from existing config file

In [ ]:
ws=Workspace.from_config()

In [ ]:
ws.get_details()

Create Azure ML Experiment

In [ ]:
exp=Experiment(workspace=ws, name='santander')

In [ ]:
Start logging metrics

In [ ]:
run=exp.start_logging()
run.log("Experiment start time", str(datetime.datetime.now()))

Load data set

In [ ]:
#load libraries
import os
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
import seaborn as sns
from ggplot import *
import gc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [ ]:
#setting the working directory
#os.chdir("G:\Edwiser material\Project\Santandarcustomer problems\Edwiser project")

In [ ]:
#Loading the required data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
#deleting ID_code variable from both test and train data
del train["ID_code"]
del test["ID_code"]

In [ ]:
# target class counting
count_class_0, count_class_1 = train.target.value_counts()

# Divide by class
train_class_0 = train[train['target'] == 0]
train_class_1 = train[train['target'] == 1]

In [ ]:
# Application of Random over sampling
class_1_over = train_class_1.sample(count_class_0, replace=True)
train_over = pd.concat([train_class_0, class_1_over], axis=0)

print('Random over-sampling:')
print(train_over.target.value_counts())

train_over.target.value_counts().plot(kind='bar', title='Count (target)');

In [ ]:
#importing Random Forest Classifier from sklearn library
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50).fit(train_over.iloc[:,1:201], train_over.iloc[:,0])

In [ ]:
filename='outputs/model.pkl'
joblib.dump(RF_Model, filename)

Test The method

In [ ]:
#filename='outputs/fare_model.pkl'
#loded_model=joblib.load(filename)

Log metrics to azure ML Experiment

In [ ]:
run.log('Intercept:',lm.intercept_)
run.log('slope:',lm.coef_[0])

End Azure ML Experiment

In [ ]:
run.log("Experiment end time", str(datetime.datetime.now()))
run.complete()

Get Portal URL

In [ ]:
print(run.get_portal_url())

Register Model

In [ ]:
model=Model.register(model_path="outputs/model.pkl",
                    model_name="santander_model",
                    tags={"key":"1"},
                    description="santander cust_transaction Prediction",
                    workspace = ws)

In [ ]:
Define Azure ML Deployment configuration

In [ ]:
aciconfig=AciWebservice.deploy_configuration(cpu_cores=1,
                                            memory_gb=1,
                                            tags={"data":"santander", "method":"sklearn"},
                                            description='Predict customer transaction')

In [ ]:
Create environment configuration file

In [ ]:
custenv=CondaDependencies()
custenv.add_conda_package("scikit-learn")

with open("custenv.yml", "w") as f:
    f.write(custenv.serialize_to_string())
with open("custenv.yml", "r")as f:
    #print(f.read())

Create Azure ML Scoring file

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    #retrive the path to the model file using the model name
    model_path=Model.get_model_path('santander_model')
    model =  joblib.load(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    #make prediction
    y_hat=model.predict(data)
    return json.dumps(y_hat.tolist())

Deploy the modelto Azure Container Instance

In [ ]:
image_config=ContainerImage.image_configuration(execution_script="score.py",
                                               runtime="python",
                                               conda_file="custenv.yml")

Explore web service

In [ ]:
service = Webservice.deploy_from_model(workspace=ws,
                                      name='santander-s',
                                      deployment_config=aciconfig,
                                      models=[model],
                                      image_config=image_config)
service.wait_for_deployment(show_output=True)

In [ ]:
service.wait_for_deployment(show_output=True)

Get the Web Service URL

In [ ]:
print(service.scoring_uri)

In [ ]:
#!curl -X POST \
#    -H 'Content-Type':'application/json'\
#    -d '{"data":[[45]]}' \
#     http://a0aada90-412a-4923-812f-ea510a5193b8.southeastasia.azurecontainer.io/score

 Delete Workspace and clean up resources

In [ ]:
# ws.delete()